In [1]:
import pandas as pd 
import numpy as np
import os
import json
import re
import torch
import ast
import cv2
from PIL import Image
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

In [181]:
path = 'Detection/annotations/tablebank_word_val.json'
path_ = re.sub('.json', '.csv', path)  


# Opening JSON file 
f = open(path,) 
  
# returns JSON object as  
# a dictionary 
data = json.load(f)

f.close()

#df = pd.read_json(path)

In [182]:
images = pd.DataFrame(data['images']).set_index('id')
annotations = pd.DataFrame(data['annotations']).set_index('id')

In [183]:
liste_images = os.listdir('Detection/images/')

In [184]:
annotations

,category_id,area,iscrowd,segmentation,image_id,bbox
id,,,,,,
5,1,101680,0,"[[85, 72, 85, 320, 495, 320, 495, 72]]",5,"[85, 72, 410, 248]"
29,1,8064,0,"[[72, 105, 72, 153, 240, 153, 240, 105]]",27,"[72, 105, 168, 48]"
106,1,22984,0,"[[90, 292, 90, 428, 259, 428, 259, 292]]",82,"[90, 292, 169, 136]"
183,1,50718,0,"[[72, 374, 72, 588, 309, 588, 309, 374]]",144,"[72, 374, 237, 214]"
313,1,225280,0,"[[70, 98, 70, 538, 582, 538, 582, 98]]",241,"[70, 98, 512, 440]"
...,...,...,...,...,...,...
101716,1,31614,0,"[[56, 502, 56, 568, 535, 568, 535, 502]]",78248,"[56, 502, 479, 66]"
101760,1,64213,0,"[[72, 209, 72, 618, 229, 618, 229, 209]]",78288,"[72, 209, 157, 409]"
101762,1,12648,0,"[[72, 444, 72, 512, 258, 512, 258, 444]]",78290,"[72, 444, 186, 68]"


In [185]:
len(liste_images)

68017

In [186]:
def right_join(row):
    if row['file_name'] not in liste_images:
        h = 0
    else:
        h = 1
    return h
        

In [187]:
images['inkl'] = images.apply(lambda row: right_join(row), axis=1)

In [188]:
images = images[images['inkl']==1]

In [189]:
images

,file_name,width,height,license,inkl
id,,,,,
3903,140919-Lpp-protokoll-2015.-aaste-riigieelarve-...,596,842,1,1
3561,1354209_21.jpg,612,792,1,1
3531,13438_1.jpg,596,842,1,1
4215,1464766196bar_gor_sev4_sunum_1.jpg,596,842,1,1
759,03_1_Izglitibas_attistibas_programma_gala_reda...,596,842,1,1
...,...,...,...,...,...
859,046_materiais_de_laboratorio_20.jpg,596,842,1,1
604,02_Fisa_generala_evaluare_M6_6B-3_24.jpg,612,792,1,1
343,0.008688001437982131_10.jpg,596,842,1,1


In [190]:
annotations
#segmentation = (x1,y1,x1,y2,x2,y2,x2,y1)
#bbox = (x1,y1,w,h)

,category_id,area,iscrowd,segmentation,image_id,bbox
id,,,,,,
5,1,101680,0,"[[85, 72, 85, 320, 495, 320, 495, 72]]",5,"[85, 72, 410, 248]"
29,1,8064,0,"[[72, 105, 72, 153, 240, 153, 240, 105]]",27,"[72, 105, 168, 48]"
106,1,22984,0,"[[90, 292, 90, 428, 259, 428, 259, 292]]",82,"[90, 292, 169, 136]"
183,1,50718,0,"[[72, 374, 72, 588, 309, 588, 309, 374]]",144,"[72, 374, 237, 214]"
313,1,225280,0,"[[70, 98, 70, 538, 582, 538, 582, 98]]",241,"[70, 98, 512, 440]"
...,...,...,...,...,...,...
101716,1,31614,0,"[[56, 502, 56, 568, 535, 568, 535, 502]]",78248,"[56, 502, 479, 66]"
101760,1,64213,0,"[[72, 209, 72, 618, 229, 618, 229, 209]]",78288,"[72, 209, 157, 409]"
101762,1,12648,0,"[[72, 444, 72, 512, 258, 512, 258, 444]]",78290,"[72, 444, 186, 68]"


In [191]:
def filename(row):
    try:
        h = images.iloc[row['image_id']-1]['file_name']
    except:
        h = '0'
    return h 

In [192]:
annotations['filename'] = annotations.apply(lambda row: filename(row), axis=1);

In [193]:
annotations = annotations[annotations['filename'] != '0']

In [194]:
annotations = annotations[['filename', 'segmentation','bbox']]
annotations.to_csv(path_,index=False)

## stacker filerne til én og laver nyt train/test split

In [75]:
annos_liste = os.listdir('Detection/annotations/')

In [76]:
annos_liste = annos_liste[1:]
annos_liste

['tablebank_latex_val.csv',
 'tablebank_latex_train.csv',
 'tablebank_latex_test.csv',
 'tablebank_word_val.csv',
 'json files',
 'skrald',
 'tablebank_word_train.csv',
 'tablebank_word_test.csv']

In [84]:
annotation_files = []
for i in annos_liste:
    if i != 'json files' and i != 'skrald':
        annotation_files.append(i)

In [85]:
annotation_files

['tablebank_latex_val.csv',
 'tablebank_latex_train.csv',
 'tablebank_latex_test.csv',
 'tablebank_word_val.csv',
 'tablebank_word_train.csv',
 'tablebank_word_test.csv']

In [86]:
path = 'Detection/annotations/'

In [87]:
Annotations_ = False
for i in range(len(annotation_files)):
    df = pd.read_csv(path + annotation_files[i])
    if Annotations_ == False:
        Annotations_ = True
        Annotations = df
        Annotations['Filename'] = re.sub('.csv', '', annotation_files[i])
    else:
        df['Filename'] = re.sub('.csv', '', annotation_files[i])
        Annotations = Annotations.append(df)

In [88]:
Annotations = Annotations.reset_index()

In [89]:
Annotations['Type'] = Annotations['Filename']


,filename,bbox
0,1401.0094_2.jpg,"[85, 445, 443, 85]"
1,1401.0190_18.jpg,"[146, 330, 311, 77]"
2,1401.0190_18.jpg,"[149, 417, 305, 77]"
3,1401.0385_5.jpg,"[154, 52, 45, 67]"
4,1401.0652_7.jpg,"[77, 117, 440, 223]"
...,...,...
83971,12360645_5.jpg,"[66, 101, 736, 400]"
83972,1482851465062-000A10458-Argentina-P115183-ABP-...,"[67, 86, 461, 108]"
83973,1-definiciones-basicas-en-proyectos-de-inversi...,"[90, 106, 427, 140]"
83974,1-definiciones-basicas-en-proyectos-de-inversi...,"[90, 273, 427, 61]"


In [97]:
Annotations = Annotations[['filename','bbox']]
Annotations.to_csv('Detection/annotations/Annotations.csv',index=False)

### Get_item

In [280]:
def boundingboxes(row):
    test = Annotations[Annotations['filename']==row['filename']]
    liste = []
    for i in range(len(test)):
        liste.append(ast.literal_eval(test.iloc[i]['bbox']))
    return liste

In [281]:
Annotations['BoundingBoxes'] = Annotations.apply(lambda row: boundingboxes(row), axis=1)

In [291]:
Annotations = Annotations[['filename', 'BoundingBoxes', 'Type']]

In [295]:
Annotations = Annotations.drop_duplicates(subset=['filename'])

In [296]:
Annotations.iloc[1]['BoundingBoxes']

[[146, 330, 311, 77], [149, 417, 305, 77]]

In [297]:
Annotations.to_csv('Detection/annotations/Annotations.csv',index=False)

In [98]:
Annotations = pd.read_csv('Detection/annotations/Annotations.csv')
Annotations

,filename,bbox
0,1401.0094_2.jpg,"[85, 445, 443, 85]"
1,1401.0190_18.jpg,"[146, 330, 311, 77]"
2,1401.0190_18.jpg,"[149, 417, 305, 77]"
3,1401.0385_5.jpg,"[154, 52, 45, 67]"
4,1401.0652_7.jpg,"[77, 117, 440, 223]"
...,...,...
83971,12360645_5.jpg,"[66, 101, 736, 400]"
83972,1482851465062-000A10458-Argentina-P115183-ABP-...,"[67, 86, 461, 108]"
83973,1-definiciones-basicas-en-proyectos-de-inversi...,"[90, 106, 427, 140]"
83974,1-definiciones-basicas-en-proyectos-de-inversi...,"[90, 273, 427, 61]"


In [99]:
msk = np.random.rand(len(Annotations)) < 0.6

train = Annotations[msk]

hep = Annotations[~msk]

In [100]:
msk = np.random.rand(len(hep)) < 0.75

test = hep[msk]

val = hep[~msk]

In [101]:
len(train)

50275

In [102]:
train.to_csv('Detection/annotations/Annotations_Train.csv',index=False)
test.to_csv('Detection/annotations/Annotations_Test.csv',index=False)
val.to_csv('Detection/annotations/Annotations_Val.csv',index=False)

In [2]:
Annotations_train = pd.read_csv('Detection/annotations/Annotations_Train.csv')
Annotations_train

,filename,BoundingBoxes,Type
0,1401.0094_2.jpg,"[[85, 445, 443, 85]]",tablebank_latex_val
1,1401.0385_5.jpg,"[[154, 52, 45, 67]]",tablebank_latex_val
2,1401.0817_13.jpg,"[[214, 522, 183, 87]]",tablebank_latex_val
3,1401.0882_3.jpg,"[[120, 323, 358, 84]]",tablebank_latex_val
4,1401.0967_19.jpg,"[[178, 130, 255, 327]]",tablebank_latex_val
...,...,...,...
40432,0.-%20PLIEGO%20TECNICAS%20GASOLEO%20DEP._6.jpg,"[[46, 415, 318, 96], [457, 79, 367, 64]]",tablebank_word_train
40433,105cchmava_10.jpg,"[[125, 248, 390, 83]]",tablebank_word_train
40434,12360645_5.jpg,"[[66, 101, 736, 400]]",tablebank_word_train
40435,1482851465062-000A10458-Argentina-P115183-ABP-...,"[[67, 86, 461, 108]]",tablebank_word_train
